In [1]:
import numpy as np
from jami import nc, input_to_grid
from pyproj import Proj
from jami.ndlinear.cinterpolate import ndlinear

In [2]:
# The input data contains the whole Australia and more
# and are in angular coordinate, i,e, Australian Albers
input_file = '../input_files/Plocsol_2x2_it3.xyz'
source_proj = Proj(init='epsg:3577')
grid_proj = Proj(init='epsg:4326')

In [3]:
# No false_easting and northing are required because the
# data are not in cartesian coordinates
source_points, source_values = input_to_grid.read_winglink_xyzv(input_file,false_northing=0.0, false_easting=0.0)

In [4]:
# grid spec will depend on the input data file
grid_spec = nc.grid_from_extent_and_resolution(left=(100.,-0.5,5),right=(190., 
-55.0, 1500), resolution=(1, -1, 50))


In [5]:
grid_x, grid_y, grid_z, grid_points = grid_spec
print(grid_x.shape, grid_y.shape, grid_z.shape)


(90,) (54,) (30,)


In [6]:
# We need to flatten the array for interpolation
flatten_transform_points = nc.transform_3d(grid_proj, source_proj, 
                            nc.flatten_grid(grid_points))


In [7]:
# change the proj of the input data into target proj
source_points_in_grid_proj = nc.transform_3d(grid_proj, source_proj, source_points)


In [ ]:
result = ndlinear(source_points_in_grid_proj, source_values,
                 flatten_transform_points)
result = result.reshape(grid_points.shape[:3])


kdtree: 0.332s


In [17]:
output_file = '../netCDF_files/seismic.nc'

In [18]:
nc.write_resistivity_grid(output_file, grid_proj, grid_y, grid_x, grid_z, 
                          result.transpose([2, 0, 1]), z_label='depth')
